In [37]:
# !pip install surprise
!which python
import sys
print(sys.path)
import numpy
print(numpy.__version__)


/opt/anaconda3/envs/surprise_env/bin/python
['/opt/anaconda3/envs/surprise_env/lib/python39.zip', '/opt/anaconda3/envs/surprise_env/lib/python3.9', '/opt/anaconda3/envs/surprise_env/lib/python3.9/lib-dynload', '', '/opt/anaconda3/envs/surprise_env/lib/python3.9/site-packages']
1.21.6


In [38]:
import pandas as pd
# 5-core-train
df = pd.read_csv('All_Beauty.train.csv')
print("shape of data = ", df.shape)
df[:5]
from surprise import KNNWithMeans, Reader, Dataset, SVD
from surprise.model_selection import train_test_split, GridSearchCV
from surprise.accuracy import rmse

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'parent_asin', 'rating']], reader)
trainset = data.build_full_trainset()

model = SVD(n_factors=50, reg_all=0.02, lr_all=0.005, n_epochs=20)
model.fit(trainset)



shape of data =  (2029, 5)


In [49]:
test_df = pd.read_csv('ALl_Beauty.test.csv')
print("shape of data = ", test_df.shape)
test_df[:5]
from surprise import KNNWithMeans, Reader, Dataset, SVD
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import accuracy

reader = Reader(rating_scale=(1, 5))
testset = list(zip(test_df['user_id'].values, test_df['parent_asin'].values, test_df['rating'].values))
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)


shape of data =  (253, 5)
RMSE: 0.9854


In [65]:
test_users = test_df['user_id']
all_items = df['parent_asin'].unique()
len(test_df)

253

In [ ]:
# print(len(predictions))
import tqdm as tqdm
def rec_k(k):
    length = len(test_users)
    count = 0
    topk = {}
    for user in test_users:
        count+=1
        candidates = []
        user_viewed = df[df["user_id"] == user]['parent_asin'].unique()
        for item in all_items:
            if item not in user_viewed:
                pred = model.predict(user, item)
                candidates.append((pred.est, item))
        candidates.sort(reverse=True)
        candidates = candidates[0:k]
        topk[user] = [item for _,item in candidates]
    # print(topk[test_users[0]])
    return topk
# print(test_df['user_id'].unique())


In [ ]:
print(test_df)

In [ ]:
# count = 0
# print(topk[test_users[0]])
# for i in tqdm.tqdm(range(len(test_users))):
#     viewed = test_df[test_df["user_id"]==test_users[i]]['rating'].unique()
#     viewed_item = test_df[test_df["user_id"]==test_users[i]]['parent_asin'].unique()[0]
#     if viewed_item not in all_items:
#         count+=1
# print(count/len(test_users))

['B08BFH35VX', 'B085SY4WC3', 'B086VYKNDF', 'B07T4LKC4D', 'B07SV4HTMC', 'B08JTNQFZY', 'B08SG2MBRY', 'B08SQZ1B3W', 'B08NJ5BTWG', 'B08CVTNQP1', 'B08B3TCMSQ', 'B08MRRNL18', 'B08N9RT9YD', 'B07ZS3DKL5', 'B08RYN11N9', 'B089ZQ8Y95', 'B08C2T24WG', 'B08QVJ4NVD', 'B08H4SYXR4', 'B07P4LLZS6', 'B09C5NQSC5', 'B08XZDXTG2', 'B088TXBCB5', 'B07M9D3WYW', 'B08SQ31H8P', 'B08L4HTQ3R', 'B081MQD52C', 'B07X8W7GJZ', 'B0914JBBTV', 'B08CZ5JZMZ', 'B08C72C56Z', 'B07JGD2T2J', 'B01EK2F1PC', 'B087F14JSH', 'B099959RFF', 'B08KY7VYDS', 'B08CC6YRLJ', 'B087PJ121M', 'B07TLMZL3T', 'B0885W8GCL', 'B07VQR3W3Z', 'B091GG4N2X', 'B0998BD871', 'B087Z9X39L', 'B08G5YVHQP', 'B08GS1G9BH', 'B0895XPZNT', 'B088LWRYJC', 'B0813ZQG3T', 'B088TGQFNM', 'B088TYPM71', 'B09BJM95J7', 'B08PVH18Z6', 'B0888CX245', 'B081D2R47W', 'B08HGZXLP6', 'B08N6YHQXT', 'B08GKVYS1Y', 'B08L6GXY2S', 'B0941YDPSW', 'B09GVHT2D3', 'B01MSF13T1', 'B0B8SFDJN9', 'B088YHN4JT', 'B07YL4485K', 'B08VKQY815', 'B086VK22QT', 'B0713VF172', 'B08TB3DZ1D', 'B08HZ5Y8RY', 'B0977LVHF8', 'B08B

100%|██████████| 253/253 [00:00<00:00, 3271.22it/s]

0.2727272727272727


In [73]:
x = []
y = []
for k in range(10, 300, 20):   
    x.append(k) 
    topk = rec_k(k)
    count = 0
    # print(topk[test_users[0]])
    for i in tqdm.tqdm(range(len(test_users))):
        viewed = test_df[test_df["user_id"]==test_users[i]]['rating'].unique()
        viewed_item = test_df[test_df["user_id"]==test_users[i]]['parent_asin'].unique()[0]
        if viewed_item in topk[test_users[i]]:
            count+=1
        # else:
        #     print(viewed_item)
        #     print(topk[test_users[i]])
    y.append(count/len(test_users))

100%|██████████| 253/253 [00:00<00:00, 2762.03it/s]


100%|██████████| 253/253 [00:00<00:00, 2853.64it/s]


100%|██████████| 253/253 [00:00<00:00, 2973.53it/s]


100%|██████████| 253/253 [00:00<00:00, 3012.29it/s]


100%|██████████| 253/253 [00:00<00:00, 3001.25it/s]


...

100%|██████████| 253/253 [00:00<00:00, 3039.40it/s]


100%|██████████| 253/253 [00:00<00:00, 2883.06it/s]


100%|██████████| 253/253 [00:00<00:00, 2968.64it/s]


100%|██████████| 253/253 [00:00<00:00, 2990.15it/s]

100%|██████████| 253/253 [00:00<00:00, 3008.86it/s]


In [74]:
print(x)
print(y)

[10, 30, 50, 70, 90, 110, 130, 150, 170, 190, 210, 230, 250, 270, 290]
[0.0, 0.05138339920948617, 0.09881422924901186, 0.12648221343873517, 0.15810276679841898, 0.1976284584980237, 0.233201581027668, 0.308300395256917, 0.383399209486166, 0.4189723320158103, 0.4980237154150198, 0.5375494071146245, 0.5810276679841897, 0.6284584980237155, 0.6600790513833992]


In [71]:
import matplotlib.pyplot as plt

AttributeError: module 'matplotlib' has no attribute 'get_data_path'

In [16]:
user_id = 'AFSKPY37N3C43SOI5IEXEK5JSIYA'
item_id = 'B07SLFWZKN'
predicted_rating = model.predict(user_id, item_id)
print(f"Estimated rating for {item_id} by {user_id} is {predicted_rating.est}")



Estimated rating for B07SLFWZKN by AFSKPY37N3C43SOI5IEXEK5JSIYA is 4.176529148815224
